# Tiền xử lý văn bản tiếng Việt

Trong notebook này, chúng ta sẽ thực hiện tiền xử lý văn bản tiếng Việt. Các bước tiền xử lý bao gồm:

1. Chuẩn hóa văn bản tiếng Việt.
2. Loại bỏ các từ không có nghĩa (stopwords).
3. Làm sạch các dấu câu thừa, URL, email, số và khoảng trắng dư thừa.
4. Xử lý và lưu dữ liệu vào các tệp CSV.
5. Sử dụng xử lý song song để tăng tốc độ khi làm việc với nhiều tệp dữ liệu.

### Cài đặt môi trường

python 3.8.10
pip install -r requirements.txt


In [1]:
# Ô 1: Import các thư viện cần thiết
import re
import os
import pandas as pd
import logging
from underthesea import word_tokenize, text_normalize
import unicodedata
from concurrent.futures import ProcessPoolExecutor
import nltk
from nltk.corpus import stopwords

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Tải stopwords tiếng Việt nếu chưa tải
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

stop_words = set([
    # Các stopwords đã định nghĩa trước đó
    "tôi", "tao", "mày", "chúng_tôi", "chúng_ta", "mình", "họ", "hắn", "nó", "chúng", "bạn",
    "là", "được", "có", "bị", "đang", "đã", "sẽ", "chưa", "đều", "phải", "cần", "muốn",
    "và", "hay", "hoặc", "nhưng", "vì", "nên", "bởi_vì", "do_đó", "vậy_nên",
    "của", "cho", "trong", "ngoài", "trên", "dưới", "với", "về", "từ", "đến",
    "rất", "khá", "thật", "còn", "đã", "vẫn", "mới", "liền", "luôn", "lại", "cứ",
    "này", "kia", "ấy", "đó", "những", "mọi", "các", "mỗi", "từng",
    "không", "chẳng", "đừng", "chả", "năng", "chưa", "chỉ", "mới",
    "một", "hai", "ba", "mấy", "vài", "nhiều", "ít",
    "hôm_nay", "ngày_mai", "hôm_qua", "bây_giờ", "lúc", "khi", "vừa",
    "cả", "tất_cả", "một_số", "một_vài", "một_chút",
    "thì", "mà", "để", "như", "là", "theo", "qua", "gì", "bởi", "vậy",
    "rằng", "rồi", "thế", "thế_là", "vẫn", "việc", "suốt", "quá",
    "cho_nên", "vì_thế", "do_vậy", "như_là", "có_thể", "phải_chi"
])


### Chuẩn hóa văn bản tiếng Việt


In [ ]:
def normalize_vietnamese_text(text):
    """
    Chuẩn hoá văn bản tiếng Việt
    """
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize('NFKC', text)
    text = text_normalize(text)
    text = re.sub(r'[„"\'`’]', ' ', text)
    text = re.sub(r'\.+', ".", text)
    text = re.sub(r'\?+', "?", text)
    text = re.sub(r'\!+', "!", text)
    return text


### Hàm loại bỏ stopwords


In [ ]:
def remove_stopwords(text):
    """
    Loại bỏ từ không có nghĩa (stopwords)
    """
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words and word.strip()]
    return " ".join(filtered_tokens)


### Tiền xử lý văn bản tiếng Việt


In [ ]:
def clean_vietnamese_text(text):
    """
    Tiền xử lý văn bản tiếng Việt: chuẩn hóa, loại bỏ stopwords, các dấu câu thừa và dữ liệu không cần thiết.
    """
    if isinstance(text, str):
        # Chuyển thành chữ thường
        text = text.lower()

        # Chuẩn hoá văn bản
        text = normalize_vietnamese_text(text)
        
        # Loại bỏ từ không có nghĩa
        text = remove_stopwords(text)
        
        # Loại bỏ cụm "dân trí" bất kể hoa hay thường
        text = re.sub(r'dân\s*trí', '', text, flags=re.IGNORECASE)
        
        # Gộp các thao tác regex: loại bỏ URL, email, số, dấu câu thừa và khoảng trắng dư thừa
        text = re.sub(r'https?://\S+|www\.\S+|\S+@\S+|\d+|[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa
        
        return text
    return ""


### Kiểm tra và tạo thư mục nếu chưa tồn tại


In [ ]:
def ensure_directory_exists(path):
    """
    Kiểm tra và tạo thư mục nếu không tồn tại
    """
    if not os.path.exists(path):
        os.makedirs(path)


### Tiền xử lý tệp văn bản đơn


In [ ]:
def preprocess_single_file(input_path):
    """
    Tiền xử lý tệp văn bản đơn và trả về văn bản đã làm sạch.
    """
    try:
        with open(input_path, 'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()
    except FileNotFoundError:
        logging.error(f"File {input_path} không tìm thấy.")
        return None
    except UnicodeDecodeError:
        logging.error(f"Lỗi giải mã file {input_path}.")
        return None

    # Làm sạch văn bản
    cleaned_text = clean_vietnamese_text(text)
    
    return cleaned_text



### Tiền xử lý nhiều tệp và lưu kết quả vào CSV


In [ ]:
def preprocess_multiple_files(categories):
    """
    Tiền xử lý nhiều tệp theo thể loại và lưu kết quả vào các tệp CSV.
    """
    for category in categories:
        input_folder = f"data/raw/{category}/"
        output_path = f"data/processed/{category}.csv"
        
        # Kiểm tra xem thư mục có tồn tại không
        if not os.path.exists(input_folder):
            logging.error(f"Thư mục {input_folder} không tồn tại.")
            continue
        
        all_cleaned_texts = []
        filenames = [filename for filename in os.listdir(input_folder) if filename.endswith('.txt')]

        # Duyệt qua tất cả các file .txt trong thư mục thể loại
        for i, filename in enumerate(filenames):
            input_path = os.path.join(input_folder, filename)
            cleaned_text = preprocess_single_file(input_path, output_path, is_last_file=(i == len(filenames)-1))
            if cleaned_text:
                all_cleaned_texts.append(cleaned_text)

        # Nếu có dữ liệu sau khi tiền xử lý, lưu vào CSV
        if all_cleaned_texts:
            ensure_directory_exists(os.path.dirname(output_path))
            df = pd.DataFrame({"cleaned_text": all_cleaned_texts})
            df.to_csv(output_path, index=False)
            logging.info(f"Đã lưu tệp đã xử lý vào {output_path}")
        else:
            logging.warning(f"Không có dữ liệu nào để lưu cho thể loại {category}.")


### Run code


In [ ]:
def main():
    categories = ['sports', 'politics', 'technology', 'health', 'lifestyle', 'law']
    
    # Tiền xử lý tệp với nhiều thể loại song song
    preprocess_multiple_files(categories)

if __name__ == "__main__":
    main()


### Kết luận

- Chúng ta đã xây dựng một quy trình tiền xử lý văn bản cho tiếng Việt với các bước như chuẩn hóa văn bản, loại bỏ stopwords và xử lý các tệp văn bản lớn bằng cách sử dụng xử lý song song.
- Bạn có thể chạy chương trình này trong môi trường của mình để tiền xử lý các tệp văn bản tiếng Việt.
